## Metodos:
Para poder concetarnos a postgre podemos crear un agente de langchain o podemos usar el archivo base de prem sql y modificaro para que fcunione con MCP
Donde MCP es Model Context Protocol que da read-only-acces a una base de datso de postgre esto se utiliza para que los LLMs puedan acceder a ella:
mas documentacion en: https://github.com/modelcontextprotocol/servers/tree/main/src/postgres



In [3]:
import pg8000
import time
from urllib.parse import urlparse
from premsql.executors.base import BaseExecutor

class PostgreSQLExecutor(BaseExecutor):
    def parse_dsn(self, dsn: str):
        parsed = urlparse(dsn)
        return {
            "user": "admin",
            "password": "gnusolidario",
            "host": "localhost",
            "port": parsed.port or 5432,
            "database": "ghdemo44"
        }

    def execute_sql(self, sql: str, dsn_or_db_path: str) -> dict:
        conn_params = self.parse_dsn(dsn_or_db_path)
        result, error = None, None
        start_time = time.time()
        
        try:
            conn = pg8000.connect(**conn_params)
            cursor = conn.cursor()
            cursor.execute(sql)
            try:
                result = cursor.fetchall()
                column_names = [desc[0] for desc in cursor.description]
            except pg8000.dbapi.ProgrammingError:
                # If no results to fetch (e.g., for INSERT)
                result = None
                column_names = None
            conn.commit()
        except Exception as e:
            error = str(e)
        finally:
            if 'cursor' in locals():
                cursor.close()
            if 'conn' in locals():
                conn.close()
            end_time = time.time()

        return {
            "result": result,
            "columns": column_names,
            "error": error,
            "execution_time": end_time - start_time,
        }


/Users/diegocarcamo/Documents/Proyecto_GNUHealth/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-14 16:39:09,518 - [UTILS] - WARNING - Unable to use transformers. Install using: pip install transformers


In [2]:
import pg8000

# Establish connection
conn = pg8000.connect(
    user="admin",
    password="gnusolidario",
    host="localhost",      # or 'db' if you're inside another Docker container
    port=5432,
    database="ghdemo44"
)

# Create a cursor to execute SQL
cursor = conn.cursor()

# Example: Run a test query
cursor.execute("SELECT * FROM account_tax_line_template WHERE id IS NOT NULL;")
tables = cursor.fetchall()

print("Tables in ghdemo44:")
for table in tables:
    print(table[0])

# Clean up
cursor.close()
conn.close()


Tables in ghdemo44:


# Execution decoding


In [ ]:
from premsql.executors import SQLiteExecutor
from premsql.generators.ollama_model import Text2SQLGeneratorOllama
from premsql.agents import BaseLineAgent
from premsql.agents.tools import SimpleMatplotlibTool

executor = SQLiteExecutor()

from premsql.generators import Text2SQLGeneratorOllama
generator = Text2SQLGeneratorOllama(
    model_name="anindya/prem1b-sql-ollama-fp116",
    experiment_name="testing_ollama",
    type="test",
)

response = generator.generate_and_save_results(
    dataset="",
    temperature=0.1,
    max_new_tokens=256,
    force=True,
    executor=executor,
    max_retries=5 # this is optional (default is already set to 5)
)

agent = BaseLineAgent(
    session_name="TEST",
    db_connection_uri="sqlite:///../california_schools.sqlite",
    specialized_model1=generator,
    specialized_model2=generator,
    executor=executor,
    plot_tool=SimpleMatplotlibTool()

)

2025-05-28 17:10:58,227 - [GENERATOR] - INFO - Experiment folder found in: /Users/diegocarcamo/Library/Caches/premsql/experiments/test/testing_ollama
Generating result ...: 0it [00:00, ?it/s]
2025-05-28 17:10:58,244 - [GENERATOR] - INFO - All responses are written to: /Users/diegocarcamo/Library/Caches/premsql/experiments/test/testing_ollama
2025-05-28 17:10:58,245 - [PIPELINE-MEMORY] - INFO - /Users/diegocarcamo/Documents/Proyecto_GNUHealth/Notebooks/premsql/premsql_pipeline_memory.db
